# Using celloracle to perform perturbation prediction

#### use celloracle perturbation prediction to predict TF perturbation effects

## Library imports

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc

from sklearn.metrics import roc_auc_score
from scipy.stats import ranksums
import scvelo as scv
from scvelo import logging as logg

import mplscience
import matplotlib.pyplot as plt
import seaborn as sns

import celloracle as co
from celloracle.applications import Gradient_calculator
from celloracle.applications import Oracle_development_module

from rgv_tools import DATA_DIR, FIG_DIR

/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_mtx from `anndata` is deprecated. Import anndata.io.read_mtx instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packag

## Function definations

In [2]:
def split_elements(character_list):
    """TODO."""
    result_list = []
    for element in character_list:
        if '_' in element:
            parts = element.split('_')
            result_list.append(parts)
        else:
            result_list.append([element])
    return result_list

def combine_elements(split_list):
    """TODO."""
    result_list = []
    for parts in split_list:
        combined_element = "_".join(parts)
        result_list.append(combined_element)
    return result_list

def pipeline(gene_for_KO):
    """TODO."""
    # 1. Simulate KO
    oracle.simulate_shift(perturb_condition={gene_for_KO: 0},
                                 ignore_warning=True,
                                 n_propagation=3)
    oracle.estimate_transition_prob(n_neighbors=n_neighbors, knn_random=True, sampled_fraction=1)
    oracle.calculate_embedding_shift(sigma_corr=0.05)
    
    ## calculate overall score
    dev = Oracle_development_module()
    # Load development flow
    dev.load_differentiation_reference_data(gradient_object=gradient)
    # Load simulation result
    dev.load_perturb_simulation_data(oracle_object=oracle, n_neighbors = n_neighbors)
    # Calculate inner product
    dev.calculate_inner_product()
    dev.calculate_digitized_ip(n_bins=10)

    # Save results in a hdf5 file.
    ps_max = dev.get_negative_PS_p_value(return_ps_sum=True, plot=False)[1]
        
    # Do simulation for all conditions.
    Lineage = []
    PS_score = []
    for lineage_name, cell_idx in index_dictionary.items():
        
        dev = Oracle_development_module()
        # Load development flow
        dev.load_differentiation_reference_data(gradient_object=gradient)
        # Load simulation result
        dev.load_perturb_simulation_data(oracle_object=oracle, cell_idx_use=cell_idx, name=lineage_name, n_neighbors = n_neighbors)
        # Calculate inner product
        dev.calculate_inner_product()
        dev.calculate_digitized_ip(n_bins=10)
        
        # Save results in a hdf5 file.
        ps = dev.get_negative_PS_p_value(return_ps_sum=True, plot=False)[1]
        ps = ps / ps_max
        Lineage.append(lineage_name)
        PS_score.append(ps)
    
    df = pd.DataFrame({"Lineage": Lineage,
                   "PS_score":PS_score})
    df.index = Lineage
    
    return df

def pipeline2(gene_for_KO):
    """TODO."""
    oracle.simulate_shift(perturb_condition={key: 0 for key in gene_for_KO},
                                 ignore_warning=True,
                                 n_propagation=3)
    oracle.estimate_transition_prob(n_neighbors=n_neighbors, knn_random=True, sampled_fraction=1)
    oracle.calculate_embedding_shift(sigma_corr=0.05)
    
    ## calculate overall score
    dev = Oracle_development_module()
    # Load development flow
    dev.load_differentiation_reference_data(gradient_object=gradient)
    # Load simulation result
    dev.load_perturb_simulation_data(oracle_object=oracle, n_neighbors = n_neighbors)
    # Calculate inner product
    dev.calculate_inner_product()
    dev.calculate_digitized_ip(n_bins=10)

    # Save results in a hdf5 file.
    ps_max = dev.get_negative_PS_p_value(return_ps_sum=True, plot=False)[1]
        
    # Do simulation for all conditions.
    Lineage = []
    PS_score = []
    for lineage_name, cell_idx in index_dictionary.items():
        
        dev = Oracle_development_module()
        # Load development flow
        dev.load_differentiation_reference_data(gradient_object=gradient)
        # Load simulation result
        dev.load_perturb_simulation_data(oracle_object=oracle, cell_idx_use=cell_idx, name=lineage_name, n_neighbors = n_neighbors)
        # Calculate inner product
        dev.calculate_inner_product()
        dev.calculate_digitized_ip(n_bins=10)
        
        # Save results in a hdf5 file.
        ps = dev.get_negative_PS_p_value(return_ps_sum=True, plot=False)[1]
        ps = ps / ps_max
        Lineage.append(lineage_name)
        PS_score.append(ps)
    
    df = pd.DataFrame({"Lineage": Lineage,
                   "PS_score":PS_score})
    df.index = Lineage
    
    return df

def TFScanning_perturbation(adata, n_states, cluster_label, terminal_states, TF):
    """TODO."""
    coef = []
    for tf in TF:
        ## TODO: mask using dynamo
        ## each time knock-out a TF
        df = pipeline(tf)
        coef.append(df.loc[:, "PS_score"])
        logg.info("Done "+tf)
    d = {'TF': TF, 'coefficient': coef}
    #df = pd.DataFrame(data=d)      
    return d

def Multiple_TFScanning_perturbation(data, n_states, cluster_label, terminal_states, TF_pair):
    """TODO."""
    coef = []
    for tf in TF_pair:
        ## TODO: mask using dynamo
        ## each time knock-out a TF
        df = pipeline2(tf)
        coef.append(df.loc[:, "PS_score"])
        logg.info("Done "+combine_elements([tf])[0])
    d = {'TF': combine_elements(TF_pair), 'coefficient': coef}
    #df = pd.DataFrame(data=d)      
    return d

def get_list_name(lst):
    """TODO."""
    names = []
    for name, obj in lst.items():
        names.append(name)
    return names

## Constants

In [3]:
DATASET = "zebrafish"

In [4]:
SAVE_DATA = True
if SAVE_DATA:
    (DATA_DIR / DATASET / "results").mkdir(parents=True, exist_ok=True)

## Data loadings

In [5]:
adata = sc.read_h5ad(DATA_DIR / DATASET / "raw" / "adata_zebrafish_preprocessed.h5ad")

## Loading SCENIC inferred GRN

In [6]:
df = pd.read_csv(DATA_DIR / DATASET / "raw" / "eRegulon_metadata_all.csv",index_col = 0)

In [7]:
df.loc[:,"Target"] = [f"{a}*{b}" for a, b in zip(df.loc[:,"Region"].tolist(), df.loc[:,"Gene"].tolist())]
df = df.loc[:,["TF","Gene","Target"]]

regulators = df['TF'].unique()
targets = df['Target'].unique()

# Create an empty binary matrix
binary_matrix = pd.DataFrame(0, columns=regulators, index=targets)

# Fill the binary matrix based on the relationships in the CSV file
for index, row in df.iterrows():
    binary_matrix.at[row['Target'], row['TF']] = 1

original_list = binary_matrix.index.tolist()
peak = [item.split('*')[0] for item in original_list]
target = [item.split('*')[1] for item in original_list]

binary_matrix.loc[:,"peak_id"] = peak
binary_matrix.loc[:,"gene_short_name"] = target

binary_matrix = binary_matrix[['peak_id', 'gene_short_name'] + [col for col in binary_matrix if col not in ['peak_id', 'gene_short_name']]]
binary_matrix = binary_matrix.reset_index(drop=True)

In [8]:
scv.pp.moments(adata, n_pcs=50, n_neighbors=30)

computing neighbors
    finished (0:00:13) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:00) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


## CellOracle processing

In [9]:
adata.X = adata.layers["matrix"].copy()
oracle = co.Oracle()
oracle.import_anndata_as_raw_count(adata=adata,
                                   cluster_column_name="cell_type",
                                   embedding_name="X_umap")

oracle.import_TF_data(TF_info_matrix=binary_matrix)

8012 genes were found in the adata. Note that Celloracle is intended to use around 1000-3000 genes, so the behavior with this number of genes may differ from what is expected.


In [10]:
oracle.perform_PCA()

In [11]:
n_comps = np.where(np.diff(np.diff(np.cumsum(oracle.pca.explained_variance_ratio_))>0.002))[0][0]

In [12]:
n_comps = min(n_comps, 50)

In [13]:
n_cell = oracle.adata.shape[0]
print(f"cell number is :{n_cell}")

k = int(0.025*n_cell)
print(f"Auto-selected k is :{k}")

cell number is :697
Auto-selected k is :17


In [14]:
oracle.knn_imputation(n_pca_dims=n_comps, k=k, balanced=True, b_sight=k*8,
                      b_maxl=k*4, n_jobs=4)

In [15]:
links = oracle.get_links(cluster_name_for_GRN_unit="cell_type", alpha=10,
                         verbose_level=10)

  0%|          | 0/13 [00:00<?, ?it/s]

Inferring GRN for NC_trunk...


  0%|          | 0/1925 [00:00<?, ?it/s]

Inferring GRN for NPB_hox3...


  0%|          | 0/1925 [00:00<?, ?it/s]

...

  0%|          | 0/1925 [00:00<?, ?it/s]

Inferring GRN for mNC_nohox...


  0%|          | 0/1925 [00:00<?, ?it/s]

Inferring GRN for mNC_vagal...


  0%|          | 0/1925 [00:00<?, ?it/s]

In [16]:
links.filter_links(p=0.001, weight="coef_abs", threshold_number=2000)
links.get_network_score()

In [17]:
links.filter_links()
oracle.get_cluster_specific_TFdict_from_Links(links_object=links)
oracle.fit_GRN_for_simulation(alpha=10,
                              use_cluster_specific_TFdict=True)

  0%|          | 0/13 [00:00<?, ?it/s]

## Define reference vector field

In [18]:
## calculate pseudotime
scv.tl.recover_dynamics(
        adata, var_names=adata.var_names, n_jobs=4
    )
scv.tl.velocity(adata, mode="dynamical")
scv.tl.latent_time(adata,min_likelihood=None)

recovering dynamics (using 4/128 cores)


  0%|          | 0/8012 [00:00<?, ?gene/s]

/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packages/a

    finished (0:04:39) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)
computing velocities
    finished (0:00:03) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
computing velocity graph (using 1/128 cores)


  0%|          | 0/697 [00:00<?, ?cells/s]

    finished (0:00:00) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)
computing terminal states
    identified 0 region of root cells and 1 region of end points .
    finished (0:00:00) --> added
    'root_cells', root cells of Markov diffusion process (adata.obs)
    'end_points', end points of Markov diffusion process (adata.obs)
computing latent time using root_cells as prior
    finished (0:00:01) --> added 
    'latent_time', shared time (adata.obs)


In [19]:
## use the velocity latent time inferred by scVelo to create gradient field
n_grid = 40
min_mass = 1.5
oracle.adata.obs["Pseudotime"] = adata.obs["latent_time"].copy()
gradient = Gradient_calculator(oracle_object=oracle, pseudotime_key="Pseudotime")
gradient.calculate_p_mass(smooth=0.8, n_grid=n_grid, n_neighbors=30)
gradient.calculate_mass_filter(min_mass=min_mass, plot=True)
gradient.transfer_data_into_grid(args={"method": "polynomial", "n_poly":3}, plot=True)
gradient.calculate_gradient()

In [20]:
scale_dev = 40
gradient.visualize_results(scale=scale_dev, s=5)

In [21]:
cell_idx_pigment = np.where(oracle.adata.obs["cell_type"].isin([
    'Pigment']))[0]

cell_idx_hox34 = np.where(oracle.adata.obs["cell_type"].isin([
    'mNC_hox34']))[0]

cell_idx_arch2 = np.where(oracle.adata.obs["cell_type"].isin([
    'mNC_arch2']))[0]

cell_idx_mesenchymal = np.where(oracle.adata.obs["cell_type"].isin([
    'mNC_head_mesenchymal']))[0]

In [22]:
index_dictionary = {"Pigment": cell_idx_pigment,
                    "mNC_hox34": cell_idx_hox34,
                    "mNC_arch2": cell_idx_arch2,
                    "mNC_head_mesenchymal": cell_idx_mesenchymal}

## Perturbation prediction

#### single knock-out

In [23]:
n_propagation = 3
n_neighbors=30

gene_list = ['elk3',
 'erf',
 'fli1a',
 'mitfa',
 'nr2f5',
 'rarga',
 'rxraa',
 'smarcc1a',
 'tfec','nr2f2']

terminal_states = [
        "mNC_head_mesenchymal",
        "mNC_arch2",
        "mNC_hox34",
        "Pigment",
    ]

In [24]:
gene_list = set(gene_list).intersection(adata.var_names)
gene_list = list(gene_list)

In [25]:
## celloracle perturbation
d = TFScanning_perturbation(adata,8,"cell_type",terminal_states,gene_list)

Done fli1a
Done rxraa
Done nr2f2
Done nr2f5
Done elk3
Done mitfa
Done tfec
Done erf
Done rarga


In [26]:
coef = pd.DataFrame(np.array(d['coefficient']))
coef.index = d['TF']
coef.columns = get_list_name(d['coefficient'][0])

#### double knock-out

In [27]:
multiple_ko = ["fli1a_elk3","mitfa_tfec","tfec_mitfa_bhlhe40","fli1a_erf_erfl3","erf_erfl3"]
multiple_ko_list = split_elements(multiple_ko)

In [28]:
d = Multiple_TFScanning_perturbation(adata,8,"cell_type",terminal_states,multiple_ko_list)

Done fli1a_elk3
Done mitfa_tfec
Done tfec_mitfa_bhlhe40
Done fli1a_erf_erfl3
Done erf_erfl3


In [31]:
coef_multiple = pd.DataFrame(np.array(d['coefficient']))
coef_multiple.index = d['TF']
coef_multiple.columns = get_list_name(d['coefficient'][0])

In [32]:
if SAVE_DATA:
    coef.to_csv(DATA_DIR / DATASET / "results" / "celloracle_perturb_single.csv")
    coef_multiple.to_csv(DATA_DIR / DATASET / "results" / 'celloracle_perturb_multiple.csv')